<a href="https://colab.research.google.com/github/obarnstedt/LINdoscope2023/blob/main/notebooks/DLC_Colab_VideoAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut Toolbox - Colab for standard (single animal) projects!
This notebook is a modified copy from [Github](https://colab.research.google.com/github/DeepLabCut/DeepLabCut/blob/master/examples/COLAB/COLAB_YOURDATA_TrainNetwork_VideoAnalysis.ipynb), originally written by Mackenzie Mathis; publication: Mathis et al 2018, https://doi.org/10.1038/s41593-018-0209-y.

It has been edited for the 2023 EMBO practical course LINdoscope: https://www.lindoscope.com.

https://github.com/DeepLabCut/DeepLabCut

![alt text](https://camo.githubusercontent.com/71523c879284afa1c51b8489491f06d24d1ff02d75dca9f71078535ee6b1978e/68747470733a2f2f696d616765732e73717561726573706163652d63646e2e636f6d2f636f6e74656e742f76312f3537663664353163396637343536366635356563663237312f313632383235303030343232392d4b565944374a4a5648594546444a33324c39564a2f444c436c6f676f323032312e6a70673f666f726d61743d3130303077)

This notebook illustrates how to use colab to:
- load a pretrained model
- analyze novel videos, filter data, create videos

This notebook demonstrates the necessary steps to use DeepLabCut for your own project. This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper.

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.

Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [1]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.3/296.3 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.6 MB/s eta 0:00:00
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=4d4c8a3159bfdfafaa803afd963785dfb5c5dbb2f3585af862fe733b221c36fe
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-

**(Be sure to click "RESTART RUNTIME" if it is displayed above above before moving on !)**

## Link your Google Drive

1. First, add a shortcut to a pretrained DLC model into you google drive. To do so, right click on the desired folder -> Organise -> Add shortcut -> My Drive -> Add

2. Next, add [a shortcut to] a folder with videos that you want to analyse to your Google Drive

3. Now, let's link to your GoogleDrive. Run the cell below and follow the authorization instructions



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import matplotlib
import os
import deeplabcut
deeplabcut.__version__

Loading DLC 2.3.5...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


'2.3.5'

## Optional: shorten videos
Add below a path to a video you want to shorten. Enter length of a desired video.

In [25]:
# PLEASE EDIT THESE:
videofile_path = '/content/drive/My Drive/LINdoscope2023_analysis/sample_data/2P_dCA1_treadmill/Basler acA780-75gm (22611479)_20190218_151620616.mp4'
length = 30 # desired length in seconds

# Edit these if needed
framerate = 75
outsuffix = '_short'

# Do not edit these
outfolder = os.path.join(os.path.dirname(videofile_path), 'DLC_'+os.path.basename(videofile_path).split('.mp4')[0])
if not os.path.exists(outfolder):
  os.makedirs(outfolder)

outpath = os.path.join(outfolder, os.path.basename(videofile_path).split('.mp4')[0]+outsuffix+'.mp4')

In [28]:
def trim_video(path_in, path_out, stop_frame, start_frame=0):
  import cv2
  from tqdm import tqdm
  cap = cv2.VideoCapture(path_in)
  cap.set(1, start_frame)
  frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = int(cap.get(cv2.CAP_PROP_FPS))
  vid_trimmed = cv2.VideoWriter(path_out, cv2.VideoWriter_fourcc(*"mp4v"),
                                fps, (frame_width, frame_height))
  vidlength = stop_frame - start_frame
  i = start_frame

  print('Trimming video...')
  with tqdm(total=vidlength) as pbar:
      while cap.isOpened():
          _, current_frame = cap.read()
          if current_frame is None or i>=stop_frame:
              break
          vid_trimmed.write(current_frame)
          pbar.update(1)
          i+=1
  cap.release()
  vid_trimmed.release()

  return path_out

In [29]:
  trim_video(videofile_path, outpath, framerate*length)

Trimming video...


100%|██████████| 2250/2250 [00:10<00:00, 210.11it/s]


'/content/drive/My Drive/LINdoscope2023_analysis/sample_data/2P_dCA1_treadmill/DLC_Basler acA780-75gm (22611479)_20190218_151620616/Basler acA780-75gm (22611479)_20190218_151620616_short.mp4'

## Setup your project variables
Modify variables **PretrainedModelFolderName** and **VideoFolderName** to names of a folder with a pretrained model and to the videos you would like to analyse, respectively.


In [30]:
# PLEASE EDIT THESE:
PretrainedModelFolderName = 'MouseBody-Barnstedt-2019-09-09'
VideoType = 'mp4'

#don't edit these:
videofile_path = [outpath] #Enter the list of videos or folder to analyze.
videofile_path

#This creates a path variable that links to your google drive copy
#No need to edit this, as you set it up before:
path_config_file = '/content/drive/My Drive/LINdoscope2023_analysis/DLC_models/'+PretrainedModelFolderName+'/config.yaml'
path_config_file

'/content/drive/My Drive/LINdoscope2023_analysis/DLC_models/MouseBody-Barnstedt-2019-09-09/config.yaml'

## Start Analyzing videos
This function analyzes your videos.

The results are stored in hd5 file in the same directory where the video resides. The data is also exported in comma-separated values format (.csv).

In [31]:
deeplabcut.analyze_videos(path_config_file, videofile_path, videotype=VideoType, shuffle=1, save_as_csv=True)

Using snapshot-1030000 for model /content/drive/My Drive/LINdoscope2023_analysis/DLC_models/MouseBody-Barnstedt-2019-09-09/dlc-models/iteration-1/MouseBodySep9-trainset95shuffle1


/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Starting to analyze %  /content/drive/My Drive/LINdoscope2023_analysis/sample_data/2P_dCA1_treadmill/DLC_Basler acA780-75gm (22611479)_20190218_151620616/Basler acA780-75gm (22611479)_20190218_151620616_short.mp4
Loading  /content/drive/My Drive/LINdoscope2023_analysis/sample_data/2P_dCA1_treadmill/DLC_Basler acA780-75gm (22611479)_20190218_151620616/Basler acA780-75gm (22611479)_20190218_151620616_short.mp4
Duration of video [s]:  30.0 , recorded with  75.0 fps!
Overall # of frames:  2250  found with (before cropping) frame dimensions:  782 582
Starting to extract posture


100%|██████████| 2250/2250 [01:29<00:00, 25.15it/s]


Saving results in /content/drive/My Drive/LINdoscope2023_analysis/sample_data/2P_dCA1_treadmill/DLC_Basler acA780-75gm (22611479)_20190218_151620616...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_resnet50_MouseBodySep9shuffle1_1030000'

You can also now filter your data to smooth any small jitters:

In [32]:
deeplabcut.filterpredictions(path_config_file, videofile_path, videotype=VideoType, shuffle=1)

Filtering with median model /content/drive/My Drive/LINdoscope2023_analysis/sample_data/2P_dCA1_treadmill/DLC_Basler acA780-75gm (22611479)_20190218_151620616/Basler acA780-75gm (22611479)_20190218_151620616_short.mp4
Saving filtered csv poses!


## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [33]:
import matplotlib
deeplabcut.plot_trajectories(path_config_file, videofile_path, videotype=VideoType, shuffle=1, filtered=True)

Loading  /content/drive/My Drive/LINdoscope2023_analysis/sample_data/2P_dCA1_treadmill/DLC_Basler acA780-75gm (22611479)_20190218_151620616/Basler acA780-75gm (22611479)_20190218_151620616_short.mp4 and data.
Plots created! Please check the directory "plot-poses" within the video directory


Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4.

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

In [34]:
deeplabcut.create_labeled_video(path_config_file,videofile_path, videotype=VideoType, shuffle=1, filtered=True)

Starting to process video: /content/drive/My Drive/LINdoscope2023_analysis/sample_data/2P_dCA1_treadmill/DLC_Basler acA780-75gm (22611479)_20190218_151620616/Basler acA780-75gm (22611479)_20190218_151620616_short.mp4
Loading /content/drive/My Drive/LINdoscope2023_analysis/sample_data/2P_dCA1_treadmill/DLC_Basler acA780-75gm (22611479)_20190218_151620616/Basler acA780-75gm (22611479)_20190218_151620616_short.mp4 and data.
Duration of video [s]: 30.0, recorded with 75 fps!
Overall # of frames: 2250 with cropped frame dimensions: 782 582
Generating frames and creating video.


100%|██████████| 2250/2250 [00:16<00:00, 140.25it/s]


[True]